# 5.3 Lab: Cross-Validation and the Bootstrap

In this lab, we explore the resampling techniques covered in chapter 5.

## 5.3.1 The Validation Set Approach

We explore the use of the validation set approach in order to estimate the
test error rates that result from fitting various linear models on the Auto
data set.
Before we begin, we use the np.random.seed() function in order to set a seed for
Python’s random number generator, so that the reader of this lab will obtain
precisely the same results as those shown below. It is generally a good idea
to set a random seed when performing an analysis such as cross-validation
that contains an element of randomness, so that the results obtained can
be reproduced precisely at a later time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model

In [2]:
import pandas as pd 
Auto = pd.read_csv('data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
                                            # "drop=True" means that you don't want it saved as a column.
Auto.shape

(392, 9)

In [3]:
Auto.shape[0]

392

In [4]:
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### Python and R use different random number generators, so we may see slightly difference results in this chapter

In [5]:
np.random.seed(1)
train = np.random.choice(Auto.shape[0], 196, replace=False) # create train data ids
select = np.in1d(range(Auto.shape[0]), train) # create a boolean array which sets TRUE to train data records
# np.in1d(): Test whether each element of a 1-D array is also present in a second array.
# Returns a boolean array the same length as ar1 that is True where an element of ar1 is in ar2 and False otherwise.


In [6]:
train

array([ 81, 165, 351, 119, 379, 236,  78,  92,  80, 333, 278, 307, 283,
       218, 366,   4, 385, 324,   6, 167, 146, 132, 120, 228,   5, 290,
       214, 197, 162, 338, 260, 232,  67, 383, 224, 185, 161, 250, 377,
        18, 229,  62, 122, 125, 106, 160, 102, 371, 189,  93,  65, 251,
       311, 389, 329, 172, 304,  17, 306, 246, 381, 180, 343, 247, 192,
       174, 207,  11, 291,  41, 318, 289, 213, 315,  23, 293,  13,  90,
        61, 334, 258, 139, 310, 349,  95, 358, 327, 294, 127, 191,  82,
       361, 222,  27,  89, 305,  73, 274, 257, 287, 107, 204,  98, 233,
       117, 277, 360, 244,  39, 159, 301, 355, 345,  59,  12, 303, 163,
        91, 332, 391, 388,  29, 273, 292,  85,  58, 354, 188, 171, 348,
       369, 298,  88, 131, 124, 230,  14, 271, 123, 138, 111,  51, 112,
         9, 175,  16, 173,   0, 105, 179, 201,  70,  38, 150, 359, 375,
       372, 145,  42, 227, 223, 208, 186, 386, 285, 272, 147, 326, 100,
        34, 110, 234, 135, 368, 154,  19, 248, 158, 267,  44, 33

In [7]:
train.sort()
train

array([  0,   4,   5,   6,   8,   9,  11,  12,  13,  14,  16,  17,  18,
        19,  23,  27,  28,  29,  34,  38,  39,  41,  42,  44,  51,  58,
        59,  61,  62,  65,  67,  70,  73,  78,  79,  80,  81,  82,  84,
        85,  88,  89,  90,  91,  92,  93,  95,  98,  99, 100, 102, 105,
       106, 107, 108, 110, 111, 112, 117, 119, 120, 122, 123, 124, 125,
       127, 131, 132, 135, 138, 139, 145, 146, 147, 150, 154, 158, 159,
       160, 161, 162, 163, 165, 167, 169, 171, 172, 173, 174, 175, 179,
       180, 185, 186, 188, 189, 191, 192, 197, 201, 204, 207, 208, 213,
       214, 218, 222, 223, 224, 225, 227, 228, 229, 230, 232, 233, 234,
       236, 238, 244, 246, 247, 248, 250, 251, 257, 258, 260, 267, 271,
       272, 273, 274, 277, 278, 283, 285, 287, 289, 290, 291, 292, 293,
       294, 298, 301, 303, 304, 305, 306, 307, 310, 311, 314, 315, 318,
       323, 324, 326, 327, 329, 331, 332, 333, 334, 338, 339, 342, 343,
       345, 348, 349, 351, 354, 355, 358, 359, 360, 361, 366, 36

In [8]:
select

array([ True, False, False, False,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False, False, False,  True, False, False, False,
        True,  True,  True, False, False, False, False,  True, False,
       False, False,  True,  True, False,  True,  True, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False,  True,  True, False,  True,  True,
       False, False,  True, False,  True, False, False,  True, False,
       False,  True, False, False, False, False,  True,  True,  True,
        True,  True, False,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False,  True, False, False,  True,
        True,  True, False,  True, False, False,  True,  True,  True,
        True, False,  True,  True,  True, False, False, False, False,
        True, False,  True,  True, False,  True,  True,  True,  True,
       False,  True,

In [9]:
print(select.shape)

(392,)


In [10]:
import statsmodels.formula.api as smf
lm = smf.ols ('mpg~horsepower', data = Auto[select]).fit()
print (lm.summary())
preds = lm.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print ('--------Test Error for 1st order--------')
print (np.mean(square_error[~select]))

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Thu, 13 Sep 2018   Prob (F-statistic):           1.28e-42
Time:                        18:15:22   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [11]:
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[select]).fit()
preds = lm2.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print ('--------Test Error for 2nd order--------')
print (np.mean(square_error[~select]))

--------Test Error for 2nd order--------
20.25269085835017


In [12]:
lm3 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = Auto[select]).fit()
preds = lm3.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print ('--------Test Error for 3rd order--------')
print (np.mean(square_error[~select]))

--------Test Error for 3rd order--------
20.325609365878517


### These results are consistent with our previous findings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favor of a model that uses a cubic function of horsepower.

### If we look at the summmary for 3rd order regression, the coefficient of the 3rd order term is not statistically significant. We can use this as Supporting evidence for the above claim. 

In [13]:
print (lm3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Thu, 13 Sep 2018   Prob (F-statistic):           4.60e-53
Time:                        18:23:00   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               66.5200 

## 5.3.2 Leave-One-Out Cross-Validation

### Trying CV in Python is not as easy as that in R. It will require some manual coding.

### To use some of implemented functions in Python, we use Sklearn for linear model 

In [14]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [15]:
x = pd.DataFrame(Auto.horsepower)
y = Auto.mpg

model = LinearRegression()
model.fit(x, y)
print (model.intercept_)
print (model.coef_)

39.93586102117047
[-0.15784473]


In [18]:
print(type(x), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


In [23]:
x.shape

(392, 1)

In [19]:
y.shape

(392,)

### K-Folds cross-validator 
provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default): http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

### cross_val_score evaluates a score by cross-validation
cv: for integer/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used.
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
    
scoring: All scorer objects follow the convention that higher return values are better than lower return values. 
Thus metrics which measure the distance between the model and the data, like metrics.mean_squared_error, 
are available as neg_mean_squared_error which return the negated value of the metric. ( -100 > -1000)
, http://scikit-learn.org/stable/modules/model_evaluation.html
    
n_jobs: The number of CPUs to use to do the computation. -1 means ‘all CPUs’.

In [ ]:
k_fold = KFold(n_splits=x.shape[0]) # Loocv use folds equal to # of observations
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
print (np.mean(-test))

In [21]:
test

array([-2.02001002e+00, -1.25092412e+00, -3.06805164e+00, -6.79901984e-02,
       -7.08255629e-01, -4.13566745e+01, -8.13755358e+01, -6.71494767e+01,
       -9.70498847e+01, -2.63430368e+01, -3.67428697e+00, -4.70741691e-01,
       -1.60507789e+00, -9.70498847e+01, -8.89557151e-01, -8.69418110e+00,
       -4.41228965e+01, -3.06562225e+01, -9.16549732e-01, -4.53185378e+01,
       -1.45705281e+00, -3.00983554e+00, -3.54617605e-03, -1.52964088e+01,
       -2.25022208e+01, -1.67905446e+01, -2.76735351e+00, -1.85354648e+01,
       -2.29957474e-01, -9.16549732e-01, -5.18379999e+00, -3.54617605e-03,
       -2.66745510e+01, -5.44791120e+01, -5.14078324e+01, -4.99405256e+01,
       -3.80360006e+01, -1.19884585e-02, -2.91033003e+00, -3.23104363e+00,
       -5.16691118e+00, -2.32487332e-01, -1.06678908e-02, -4.82615264e-01,
       -2.10211114e+01, -4.35585197e+01, -2.66745510e+01, -6.51231162e+01,
       -1.13690418e+01, -5.18379999e+00, -1.25085727e+00, -4.27873211e+00,
       -1.77161819e+00, -

In [22]:
-test

array([2.02001002e+00, 1.25092412e+00, 3.06805164e+00, 6.79901984e-02,
       7.08255629e-01, 4.13566745e+01, 8.13755358e+01, 6.71494767e+01,
       9.70498847e+01, 2.63430368e+01, 3.67428697e+00, 4.70741691e-01,
       1.60507789e+00, 9.70498847e+01, 8.89557151e-01, 8.69418110e+00,
       4.41228965e+01, 3.06562225e+01, 9.16549732e-01, 4.53185378e+01,
       1.45705281e+00, 3.00983554e+00, 3.54617605e-03, 1.52964088e+01,
       2.25022208e+01, 1.67905446e+01, 2.76735351e+00, 1.85354648e+01,
       2.29957474e-01, 9.16549732e-01, 5.18379999e+00, 3.54617605e-03,
       2.66745510e+01, 5.44791120e+01, 5.14078324e+01, 4.99405256e+01,
       3.80360006e+01, 1.19884585e-02, 2.91033003e+00, 3.23104363e+00,
       5.16691118e+00, 2.32487332e-01, 1.06678908e-02, 4.82615264e-01,
       2.10211114e+01, 4.35585197e+01, 2.66745510e+01, 6.51231162e+01,
       1.13690418e+01, 5.18379999e+00, 1.25085727e+00, 4.27873211e+00,
       1.77161819e+00, 3.58044876e+01, 1.21519855e+01, 8.41044041e+00,
      

### For higher order polynomial fit, we use pipline tool (http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Below shows how to fit an order 1 to 5 polynomial data and show the loocv results
PolynomialFeatures: Generate polynomial and interaction features. For example, if an input sample is two dimensional and of the form [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].

In [23]:
A = []
for porder in range(1, 6):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=x.shape[0]) # loocv use folds equal to # of observations
    test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    A.append(np.mean(-test))
    
print (A)

[24.231513517929226, 19.24821312448941, 19.33498406411397, 19.424430309374937, 19.03322024895203]


## 5.3.3 k-Fold Cross-Validation

### K-fold validation is exactly same as LOOCV with different n_splits parameter setup. The computation time is much shorter than that of LOOCV.

In [24]:
np.random.seed(3)
A = []
for porder in range(1, 11):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=10) 
    test = cross_val_score(model, x, y, cv = k_fold,  scoring = 'neg_mean_squared_error', n_jobs = -1)
    A.append(np.mean(-test))
    
print (A)

[27.439933652339857, 21.235840055802118, 21.336606183328527, 21.35388699303018, 20.90563774089812, 20.80193369142537, 20.95328706780984, 21.07715570228614, 21.03690306486462, 20.980936037598624]


### We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## 5.3.4 The Bootstrap

### Bootstrap means sampling with replacement. To eliminate the effect of sample size, the norm practice is to sample the same size as original dataset with replacement.

In [25]:
Portfolio = pd.read_csv('data/Portfolio.csv', header=0)

In [26]:
Portfolio.shape

(100, 2)

In [27]:
Portfolio

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
5,-1.737124,-2.037191
6,1.966413,1.452957
7,2.152868,-0.434139
8,-0.081208,1.450809
9,-0.891782,0.821016


### To illustrate the use of the bootstrap on this data, we must first create a function, alpha_fn(), which takes as input the (X, Y) data as well as a vector indicating which observations should be used to estimate alpha.

In [28]:
def alpha_fn(data, index):
    X = data.X[index]
    Y = data.Y[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

In [29]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [34]:
type(np.arange(0, 100))

numpy.ndarray

In [30]:
alpha_fn(Portfolio, np.arange(0, 100))

0.5766511516104118

In [32]:
np.cov(Portfolio.X,Portfolio.Y)

array([[1.12864236, 0.62635829],
       [0.62635829, 1.30823747]])

In [31]:
np.cov(Portfolio.X,Portfolio.Y)[0,1]

0.6263582921063724

### Generate one set of random index with 100 elements. The array has been sorted to show there are repeat elements.

In [35]:
np.sort(np.random.choice(range(0, 100), size=100, replace=True))

array([ 0,  0,  1,  1,  2,  2,  3,  7,  9, 10, 13, 14, 16, 16, 18, 18, 19,
       20, 20, 21, 21, 21, 21, 22, 24, 24, 26, 28, 28, 29, 29, 30, 32, 32,
       33, 33, 33, 33, 36, 37, 37, 37, 37, 38, 39, 39, 41, 43, 44, 44, 46,
       48, 48, 48, 48, 49, 51, 52, 54, 54, 55, 55, 56, 56, 56, 56, 59, 60,
       60, 61, 62, 63, 63, 64, 66, 69, 71, 72, 74, 74, 74, 75, 78, 79, 80,
       81, 85, 88, 90, 90, 90, 91, 91, 93, 94, 95, 96, 96, 97, 99])

### Recall the previous function with a random set of input. 

In [36]:
alpha_fn(Portfolio, np.random.choice(range(0, 100), size=100, replace=True))

0.5699422338741068

### Define an ad hoc function called boot_python()

In [37]:
def boot_python(data, input_fun, iteration):
    n = Portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n)) # Return random integers from low (inclusive) to high (exclusive).
                                                  # size: output shape, e.g., (1000, 100)
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_fun(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}
    

In [38]:
boot_python(Portfolio, alpha_fn, 1000)

{'Mean': 0.5783005990543031, 'STD': 0.0912134132867913}

In [39]:
n = 100
iteration = 1000
idx = np.random.randint(0, n, (iteration, n))
idx

array([[62, 56, 83, ..., 49, 51,  0],
       [16, 62,  3, ...,  5, 93, 52],
       [95, 68, 13, ..., 69, 78, 50],
       ...,
       [ 7, 94, 59, ..., 80, 22, 11],
       [40, 11, 39, ...,  1, 52, 60],
       [96, 55, 79, ..., 75, 96, 95]])

In [40]:
len(idx)

1000

In [41]:
np.zeros(1000)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

### Similar idea can be used in a lot of other places, such as estimating the accuracy of a linear regression model coeffcients.